## Import libs

In [1]:
#Import libs 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statistics as st
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
import seaborn as sns
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
import random
import mlflow
import mlflow.sklearn
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 15)

## Upload main data

In [2]:
# # Loading the ACG_wells_JOINT_BEST_v6.csv file
# path = 'C:\\jupyter\\SPP\\input\\'
# data_init = pd.read_csv(path + 'ACG_wells_JOINT_BEST_v10.csv', sep=',')
# df = data_init.copy()
# df = df[1:]

In [6]:
# #Select only neccessary data
# df_cln = df[[   'wellName', 'MD', 'BADPORLOG', 'Casings', 'FORMATION', 'DEVI', 'HAZI',
#                 'FLANK', 'FLANK1', 'FLANK2', 'NET', 'NET_VSH','FLUIDS',
#                 'LPERM', 'PHIT', 
#                 'GR_N', 'GRMATRIX', 'GRSHALE','VSH', 'NPSS', 'RHOB', 'RHOF', 'RHOMA', 
#                 'RDEEP',  'SON', 'SONSH', 
#                 'TVD_SCS','TST', 'X', 'Y']]
# #Fill up nan and -9999 values with 0
# df_cln = df_cln.fillna(0)
# df_cln = df_cln.replace(-9999, 0)
# df_cln = df_cln.replace('-9999', '0')
# #Assing proper datatypes for df
# dicttypes = {   'wellName':'string', 'MD':'float', 'BADPORLOG':'int', 'Casings':'float', 'FORMATION':'string','DEVI':'float', 'HAZI':'float',
#                 'FLANK':'int', 'FLANK1':'int', 'FLANK2':'int', 'NET':'int', 'NET_VSH':'int', 'FLUIDS':'int',
#                 'LPERM':'float','PHIT':'float',
#                 'GR_N':'float', 'GRMATRIX':'float', 'GRSHALE':'float', 'VSH':'float', 'NPSS':'float','RHOB':'float','RHOF':'float', 'RHOMA':'float',
#                 'RDEEP':'float', 'SON':'float', 'SONSH':'float',
#                 'TVD_SCS':'float', 'TST':'float', 'X':'float', 'Y':'float'}
# df_cln = df_cln.astype(dicttypes, errors='ignore')
# df_cln.loc[df_cln.FORMATION=='0', 'FORMATION']='None'
# # Save data to parquet
# df_cln.to_parquet('ACG_wells_JOINT_BEST_v10.parquet.gzip', compression='gzip')

In [14]:
#Loading metadata, distribution wells per Platforms and all the that.
path = 'C:\\jupyter\\SPP\\input\\'
metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
metadata = metadata_init.copy()
metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
metadata.Status = metadata.Status.str.strip()
metadata.Status = metadata.Status.str.lower()
metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

#Read data from parquet
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
df_prq.rename(columns={'wellName':'well'}, inplace=True)
df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
# print('wells in df totally:', len(df_prq.well.unique()))
# Filter data with bad_well_list 
bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP']
df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
#Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
#Getting XY coords of Balakhany formation tops
xy_coord = df_prq[['well', 'FORMATION', 'X', 'Y']]
xy_coord = xy_coord.groupby(['well', 'FORMATION']).apply(lambda x: x.iloc[0]).drop(columns=['well', 'FORMATION']).reset_index()
xy_coord = xy_coord[xy_coord.FORMATION.str.contains('Balakhany') & (xy_coord.X>0) & (xy_coord.Y>0)]
#Find top TVD_SCS for each formation
df_prq_tvdss = df_prq[['well','MD','FORMATION','TVD_SCS']].groupby(['well','FORMATION']).apply(lambda x: x.iloc[0])
df_prq_tvdss = df_prq_tvdss.drop(['well','FORMATION'], axis=1).reset_index()
df_prq_tvdss = df_prq_tvdss[df_prq_tvdss.TVD_SCS>0]

## Data preparation

### Cleaning dataset for outliers

In [18]:
#Reading csv with initial KHtst_v3, joining xy-coord & TVD_SCS tops of formation
path = 'C:\\jupyter\\SPP\\inputoutput\\'
df_khtst = pd.read_csv(path + 'df_prq_khtst_v3.csv')
df_khtst_xy = df_khtst.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_khtst_xy_tvd = df_khtst_xy.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_khtst_xy_tvd_fld = df_khtst_xy_tvd.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
#Clean dataset for outliers for Balakhany VIII & X  for AZR and CHG fields by rule 1.5 * IQR
fm_list_8_10 = ['Balakhany VIII', 'Balakhany VIII sand', 'Balakhany VIII 25','Balakhany VIII 20', 
                'Balakhany VIII 15', 'Balakhany VIII 10', 'Balakhany VIII 5',
                'Balakhany X', 'Balakhany X sand', 'Balakhany X 40', 'Balakhany X 20'] 
azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI']
chg_lst = ['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG']
df_lst = []
for fm in fm_list_8_10:
    df_khtst_fm = df_khtst_xy_tvd_fld[(df_khtst_xy_tvd_fld.FORMATION == fm) & (df_khtst_xy_tvd_fld.field.isin(azr_lst))]
    Q1 = df_khtst_fm['KHtst'].quantile(0.25)
    Q3 = df_khtst_fm['KHtst'].quantile(0.75)
    IQR = Q3 - Q1
    # print(f'bal {fm} azr IQR', IQR, 'bot limit:', (Q1 - 1.5 * IQR), 'top limit:', (Q3 + 1.5 * IQR))
    df_khtst_fm_qcl = df_khtst_fm[~((df_khtst_fm['KHtst'] < (Q1 - 1.5 * IQR)) | (df_khtst_fm['KHtst'] > (Q3 + 1.5 * IQR)))]
    df_lst.append(df_khtst_fm_qcl)
for fm in fm_list_8_10:
    df_khtst_fm = df_khtst_xy_tvd_fld[(df_khtst_xy_tvd_fld.FORMATION == fm) & (df_khtst_xy_tvd_fld.field.isin(chg_lst))]
    Q1 = df_khtst_fm['KHtst'].quantile(0.25)
    Q3 = df_khtst_fm['KHtst'].quantile(0.75)
    IQR = Q3 - Q1
    # print(f'bal {fm} chg IQR', IQR, 'bot limit:', (Q1 - 1.5 * IQR), 'top limit:', (Q3 + 1.5 * IQR))
    df_khtst_fm_qcl = df_khtst_fm[~((df_khtst_fm['KHtst'] < (Q1 - 1.5 * IQR)) | (df_khtst_fm['KHtst'] > (Q3 + 1.5 * IQR)))]
    df_lst.append(df_khtst_fm_qcl)
df_khtst_bal_qcl = pd.concat(df_lst)

## AllFeatures dataset

### List of functions

In [19]:
#Uploading k_htst data from csv-file & Calculation of Euclidean Distances
path = 'C:\\jupyter\\SPP\\inputoutput\\'
df_khtst = pd.read_csv(path + 'df_prq_khtst_v3.csv')
df_khtst_xy = df_khtst.set_index(['well','FORMATION']).join(
                                                            df_prq[['well','FORMATION','X','Y','TVD_SCS']].groupby(
                                                            ['well','FORMATION']).apply(lambda x: x.iloc[0]).drop(
                                                            ['well','FORMATION'], axis=1)
                                                            ).reset_index()
def well_display(wellname, fmname):
    data = df_prq[(df_prq.well==wellname) & (df_prq.FORMATION == fmname)]
    depth = data['MD']
    grn = data['GR_N']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    fluid = data['gross_pay']
    perm = data['LPERM']
    fig, ax = plt.subplots(1,5, figsize=(7,7), sharey=True)
    ax[0].plot(grn, depth, color='lightgreen'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150)
    ax[1].plot(rhob, depth, color='red'), ax[1].invert_yaxis(), ax[1].set_xlim(1.65, 2.65)
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue')
    twin1.set_xlim(0.6, 0)
    ax[2].plot(rdeep, depth, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(1, 100), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[3].plot(phit, depth, color='green'), ax[3].set_xlim(0.3, 0), ax[3].grid(axis='x'), ax[3].invert_yaxis()
    ax[3].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[3].twiny()
    twin2.plot(fluid, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,fluid, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    ax[4].plot(perm, depth, color='purple'), ax[4].set_xscale('log'), ax[4].set_xlim(1, 1000), ax[4].grid(axis='x'), ax[4].invert_yaxis()
    fig.suptitle(wellname + ' ' + fmname, fontsize=14)
    fig.tight_layout()
#Calculation of Euclidean Distances for the top of Balakhany VIII sand & Balakhany X sand
def well_dist_calc(dist_formation, iqr_clean, iqr_wells):
    """
    dist_formation, iqr_clean=f(1,0), iqr_wells == df_khtst_bal_qcl
    """
    data = df_khtst_xy_tvd[(df_khtst_xy_tvd.FORMATION == dist_formation) & (df_khtst_xy_tvd.X > 0) & (df_khtst_xy_tvd.Y > 0) &
                            (df_khtst_xy_tvd.MD.notna())]
    if iqr_clean == 1:
        data = data[data.well.isin(iqr_wells)]
    else:
        pass
    row_name = data.well.reset_index().drop(['index'], axis=1)
    distance_fm = pd.DataFrame(euclidean_distances(data[['X', 'Y', 'TVD_SCS']]), columns=list(data.well))
    distance_fm_well = distance_fm.join(row_name).set_index('well')
    distance_fm_well.reset_index()
    return distance_fm_well.reset_index()
#Collecting of EucDist, XY and KHtst data 
def well_kh_accum(data, kh_formation):
    well_kh_accum = []
    well_x_accum = []
    well_y_accum = []
    for i in data:
        well_kh_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['KHtst'].reset_index())    
        well_x_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['X'].reset_index())
        well_y_accum.append(df_khtst_xy[(df_khtst_xy.well==i)&(df_khtst_xy.FORMATION == kh_formation)]['Y'].reset_index())
    well_kh3 = pd.concat(well_kh_accum).T[1:]
    well_kh3.columns = ['kh1','kh2','kh3']
    well_x3 = pd.concat(well_x_accum).T[1:]
    well_x3.columns = ['x1','x2','x3']
    well_y3 = pd.concat(well_y_accum).T[1:]
    well_y3.columns = ['y1','y2','y3']
    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                        well_x3.reset_index().drop('index',axis=1), 
                        well_y3.reset_index().drop('index',axis=1)], axis=1)
    return final
#Collecting of Dist & Wells data togher with func(well_kh_accum)
def well_offsets_collection(dist_dataset, off_formation, kh_dataset, metadata):
    df_collect = []
    for num, well_name in enumerate(dist_dataset.well[:]):
        well_dist3 = dist_dataset[dist_dataset.well == well_name].T[1:].sort_values(by=num)[1:4].reset_index()
        well_dist3_tuple = tuple(well_dist3['index'])
        well_dist3_res = well_dist3.T[1:].reset_index().drop('index', axis=1)   
        well_name3_res = well_dist3.T[:1].reset_index().drop('index', axis=1)
        well_kh3_res = well_kh_accum(well_dist3_tuple, off_formation)
        well_dist3_res.columns =['dist1', 'dist2', 'dist3']
        well_name3_res.columns =['well1', 'well2', 'well3']
        concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
        result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
        df_collect.append(result)       
    df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
    df_khtst_xy_bal = kh_dataset[kh_dataset.FORMATION==off_formation][['well', 'FORMATION', 'KHtst']]
    df_well_kh_dist_frm = df_well_kh_dist.set_index('well').join(df_khtst_xy_bal.set_index('well')).reset_index()
    df_well_kh_dist_frm_fld = df_well_kh_dist_frm.set_index('well').join(metadata[['well','field']].set_index('well')).reset_index()
    return df_well_kh_dist_frm_fld
#Calculating avg properties and collecting df per FU
def df_avg_properties(formation_avg_prop, df_avg_prop, df_joined):
    df_hpv = df_joined[df_joined.FORMATION == formation_avg_prop][['well','FORMATION','res_tst','phit_wavg','vsh_wavg']]
    df_permh = df_avg_prop[df_avg_prop.FORMATION == formation_avg_prop].groupby(['well','FORMATION'])['kavg_htst'].sum().reset_index()
    df_phhpv = df_hpv.set_index(['well','FORMATION']).join(df_permh.set_index(['well','FORMATION'])).reset_index()
    return df_phhpv
#Rotate x,y around xo,yo by theta (rad)
def rotate(x,y): 
    theta = (math.pi/180)*34
    xo = st.median(np.array(df_khtst_xy['X'].to_list()))
    yo = st.median(np.array(df_khtst_xy['Y'].to_list()))
    xr = math.cos(theta)*(x-xo)-math.sin(theta)*(y-yo) + xo
    yr = math.sin(theta)*(x-xo)+math.cos(theta)*(y-yo) + yo
    return [xr,yr]

In [20]:
def rfr_train_test_split(train_dataset):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well','kavg_htst']])
    x = np.array(train_dataset.drop('kavg_htst', axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop('kavg_htst', axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    x_train_wells = x_train_init[:,1]
    x_test_wells = x_test_init[:,1]
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_train_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    grid_rfr = RandomForestRegressor()
    grid_param_rfr = {
                        'bootstrap': [True],
                        'max_depth': [2,10,40,100],
                        'min_samples_leaf': [1,2,5,10],
                        'min_samples_split': [1,2,5,10],
                        'n_estimators': [50,75,125]}
    scorer = make_scorer(r2, greater_is_better=True)
    grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = grid_param_rfr, scoring=scorer, cv = 15)
    grid_calc_rfr.fit(x_train, y_train)
    gd_sr_setting = grid_calc_rfr.best_params_
    grids_setting_list.append(gd_sr_setting)
    # Applying Pipeline for ML-model
    rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(   bootstrap= gd_sr_setting['bootstrap'], 
                                                                                max_depth=gd_sr_setting['max_depth'],
                                                                                min_samples_leaf=gd_sr_setting['min_samples_leaf'], 
                                                                                min_samples_split=gd_sr_setting['min_samples_split'], 
                                                                                n_estimators=gd_sr_setting['n_estimators']))])
    rfr.fit(x_train, y_train)
    y_pred_train = rfr.predict(x_train)
    y_pred_test = rfr.predict(x_test)
    # Reporting
    df_rfr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_rfr_train['l_limit'] = df_rfr_train.actual*0.75
    df_rfr_train['h_limit'] = df_rfr_train.actual*1.25
    df_rfr_train['qc'] = 'out'
    df_rfr_train['dataset'] = 'train'
    df_rfr_train.loc[(df_rfr_train.predict >= df_rfr_train.l_limit) & (df_rfr_train.predict <= df_rfr_train.h_limit), 'qc'] = 'in'
    df_rfr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_rfr_test['l_limit'] = df_rfr_test.actual*0.75
    df_rfr_test['h_limit'] = df_rfr_test.actual*1.25
    df_rfr_test['qc'] = 'out'
    df_rfr_test['dataset'] = 'test'
    df_rfr_test.loc[(df_rfr_test.predict >= df_rfr_test.l_limit) & (df_rfr_test.predict <= df_rfr_test.h_limit), 'qc'] = 'in'
    df_rfr_result = pd.concat([df_rfr_train,df_rfr_test])

    metrics_dict = {    'r2_train':     r2(y_train, y_pred_train).round(2), 
                        'r2_test':      r2(y_test, y_pred_test).round(2),
                        'mae_train':    mae(y_train, y_pred_train).round(2), 
                        'mae_test':     mae(y_test, y_pred_test).round(2),
                        'test_in':      df_rfr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict, 'grid_search':grids_setting_list, 'result_df':df_gbr_result}
def rfr_train_test_split_iter(train_dataset,target_feature):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well',target_feature]])
    x = np.array(train_dataset.drop(target_feature, axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop(target_feature, axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    x_train_wells = x_train_init[:,1]
    x_test_wells = x_test_init[:,1]
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_train_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    grid_rfr = RandomForestRegressor()
    grid_param_rfr = {
                        'bootstrap': [True],
                        'max_depth': [10],
                        'min_samples_leaf': [2],
                        'min_samples_split': [5],
                        'n_estimators': [50]}
    # scorer = make_scorer(r2, greater_is_better=True)
    # grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = grid_param_rfr, scoring=scorer, cv = 15)
    # grid_calc_rfr.fit(x_train, y_train)
    # gd_sr_setting = grid_calc_rfr.best_params_
    # grids_setting_list.append(gd_sr_setting)
    # Applying Pipeline for ML-model
    rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(   bootstrap= grid_param_rfr['bootstrap'][0], 
                                                                                max_depth=grid_param_rfr['max_depth'][0],
                                                                                min_samples_leaf=grid_param_rfr['min_samples_leaf'][0], 
                                                                                min_samples_split=grid_param_rfr['min_samples_split'][0], 
                                                                                n_estimators=grid_param_rfr['n_estimators'][0]))])
    rfr.fit(x_train, y_train)
    y_pred_train = rfr.predict(x_train)
    y_pred_test = rfr.predict(x_test)
    # Reporting
    df_rfr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_rfr_train['l_limit'] = df_rfr_train.actual*0.75
    df_rfr_train['h_limit'] = df_rfr_train.actual*1.25
    df_rfr_train['qc'] = 'out'
    df_rfr_train['dataset'] = 'train'
    df_rfr_train.loc[(df_rfr_train.predict >= df_rfr_train.l_limit) & (df_rfr_train.predict <= df_rfr_train.h_limit), 'qc'] = 'in'
    df_rfr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_rfr_test['l_limit'] = df_rfr_test.actual*0.75
    df_rfr_test['h_limit'] = df_rfr_test.actual*1.25
    df_rfr_test['qc'] = 'out'
    df_rfr_test['dataset'] = 'test'
    df_rfr_test.loc[(df_rfr_test.predict >= df_rfr_test.l_limit) & (df_rfr_test.predict <= df_rfr_test.h_limit), 'qc'] = 'in'
    df_rfr_result = pd.concat([df_rfr_train,df_rfr_test])

    metrics_dict = {    'r2_train':     r2(y_train, y_pred_train).round(2), 
                        'r2_test':      r2(y_test, y_pred_test).round(2),
                        'mae_train':    mae(y_train, y_pred_train).round(2), 
                        'mae_test':     mae(y_test, y_pred_test).round(2),
                        'test_in':      df_rfr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict, 'grid_search':grids_setting_list, 'result_df':df_rfr_result}

In [21]:
def svr_train_test_split(train_dataset):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well','kavg_htst']])
    x = np.array(train_dataset.drop('kavg_htst', axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop('kavg_htst', axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    x_train_wells = x_train_init[:,1]
    x_test_wells = x_test_init[:,1]
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_train_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    grid_svr = SVR()
    grid_param_svr = {'kernel' : (['rbf']),
                    'C' : [10, 100, 500, 1000, 2000, 3000],
                    'gamma':[0.005, 0.01, 0.5],
                    'epsilon': [0.001,0.01, 1, 5]}
    scorer = make_scorer(r2, greater_is_better=True)
    grid_calc_svr = GridSearchCV(estimator = grid_svr, param_grid = grid_param_svr, scoring=scorer, cv = 15)
    grid_calc_svr.fit(x_train, y_train)
    gd_sr_setting = grid_calc_svr.best_params_
    grids_setting_list.append(gd_sr_setting)
    # Applying Pipeline for ML-model
    svr = Pipeline([("scaler",StandardScaler()),("svr",SVR( kernel = 'rbf', 
                                                            C=gd_sr_setting['C'], 
                                                            gamma = gd_sr_setting['gamma'], 
                                                            epsilon=gd_sr_setting['epsilon']))])
    svr.fit(x_train, y_train)
    y_pred_train = svr.predict(x_train)
    y_pred_test = svr.predict(x_test)
    # Reporting
    df_svr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_svr_train['l_limit'] = df_svr_train.actual*0.75
    df_svr_train['h_limit'] = df_svr_train.actual*1.25
    df_svr_train['qc'] = 'out'
    df_svr_train['dataset'] = 'train'
    df_svr_train.loc[(df_svr_train.predict >= df_svr_train.l_limit) & (df_svr_train.predict <= df_svr_train.h_limit), 'qc'] = 'in'
    df_svr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_svr_test['l_limit'] = df_svr_test.actual*0.75
    df_svr_test['h_limit'] = df_svr_test.actual*1.25
    df_svr_test['qc'] = 'out'
    df_svr_test['dataset'] = 'test'
    df_svr_test.loc[(df_svr_test.predict >= df_svr_test.l_limit) & (df_svr_test.predict <= df_svr_test.h_limit), 'qc'] = 'in'
    df_svr_result = pd.concat([df_svr_train,df_svr_test])

    metrics_dict = {    'r2_train':     r2(y_train, y_pred_train).round(2), 
                        'r2_test':      r2(y_test, y_pred_test).round(2),
                        'mae_train':    mae(y_train, y_pred_train).round(2), 
                        'mae_test':     mae(y_test, y_pred_test).round(2),
                        'test_in':      df_svr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict, 'grid_search':grids_setting_list, 'result_df':df_gbr_result}
def svr_train_test_split_iter(train_dataset,target_feature):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset = df_allf
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well',target_feature]])
    x = np.array(train_dataset.drop(target_feature, axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop(target_feature, axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    x_train_wells = x_train_init[:,1]
    x_test_wells = x_test_init[:,1]
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_train_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    # 'grid_search': [{'C': 3000, 'epsilon': 5, 'gamma': 0.005, 'kernel': 'rbf'}]
    grid_svr = SVR()
    grid_param_svr = {  'C' : [3000],
                        'gamma':[0.005],
                        'epsilon': [5]}
    # Applying Pipeline for ML-model
    svr = Pipeline([("scaler",StandardScaler()),("svr", SVR( kernel = 'rbf', 
                                                            C=grid_param_svr['C'][0], 
                                                            gamma = grid_param_svr['gamma'][0], 
                                                            epsilon=grid_param_svr['epsilon'][0]))])
    svr.fit(x_train, y_train)
    y_pred_train = svr.predict(x_train)
    y_pred_test = svr.predict(x_test)
    # Reporting
    df_svr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_svr_train['l_limit'] = df_svr_train.actual*0.75
    df_svr_train['h_limit'] = df_svr_train.actual*1.25
    df_svr_train['qc'] = 'out'
    df_svr_train['dataset'] = 'train'
    df_svr_train.loc[(df_svr_train.predict >= df_svr_train.l_limit) & (df_svr_train.predict <= df_svr_train.h_limit), 'qc'] = 'in'
    df_svr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_svr_test['l_limit'] = df_svr_test.actual*0.75
    df_svr_test['h_limit'] = df_svr_test.actual*1.25
    df_svr_test['qc'] = 'out'
    df_svr_test['dataset'] = 'test'
    df_svr_test.loc[(df_svr_test.predict >= df_svr_test.l_limit) & (df_svr_test.predict <= df_svr_test.h_limit), 'qc'] = 'in'
    df_svr_result = pd.concat([df_svr_train,df_svr_test])

    metrics_dict_svr = {'r2_train':r2(y_train, y_pred_train).round(2), 
                        'r2_test': r2(y_test, y_pred_test).round(2),
                        'mae_train':mae(y_train, y_pred_train).round(2), 
                        'mae_test': mae(y_test, y_pred_test).round(2),
                        'test_in': df_svr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict_svr, 'result_df':df_svr_result}

In [22]:
def gbr_train_test_split(train_dataset):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well','kavg_htst']])
    x = np.array(train_dataset.drop('kavg_htst', axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop('kavg_htst', axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_test_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    grid_gbr = GradientBoostingRegressor()
    grid_param_gbr = {  "n_estimators": [1, 5, 10, 50],
                        "max_depth": [1, 10, 50, 100],
                        "learning_rate": [0.001, 0.1, 1],
                        "max_features": ["sqrt", "log2"],
                        "random_state": [42, 90]}
    scorer = make_scorer(r2, greater_is_better=True)
    grid_calc_gbr = GridSearchCV(estimator = grid_gbr, param_grid = grid_param_gbr, scoring=scorer, cv = 5)
    grid_calc_gbr.fit(x_train, y_train)
    gd_sr_setting = grid_calc_gbr.best_params_
    grids_setting_list.append(gd_sr_setting)
    # Applying for ML-model
    gbr = Pipeline([("scaler",StandardScaler()),("gbr",GradientBoostingRegressor(   n_estimators = gd_sr_setting['n_estimators'], 
                                                                                    max_depth = gd_sr_setting['max_depth'],  
                                                                                    learning_rate = gd_sr_setting['learning_rate'], 
                                                                                    max_features = gd_sr_setting['max_features'],
                                                                                    random_state = gd_sr_setting['random_state']))]) 
    gbr.fit(x_train, y_train)
    y_pred_train = gbr.predict(x_train)
    y_pred_test = gbr.predict(x_test)
    # Reporting
    df_gbr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_gbr_train['l_limit'] = df_gbr_train.actual*0.75
    df_gbr_train['h_limit'] = df_gbr_train.actual*1.25
    df_gbr_train['qc'] = 'out'
    df_gbr_train['dataset'] = 'train'
    df_gbr_train.loc[(df_gbr_train.predict >= df_gbr_train.l_limit) & (df_gbr_train.predict <= df_gbr_train.h_limit), 'qc'] = 'in'
    df_gbr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_gbr_test['l_limit'] = df_gbr_test.actual*0.75
    df_gbr_test['h_limit'] = df_gbr_test.actual*1.25
    df_gbr_test['qc'] = 'out'
    df_gbr_test['dataset'] = 'test'
    df_gbr_test.loc[(df_gbr_test.predict >= df_gbr_test.l_limit) & (df_gbr_test.predict <= df_gbr_test.h_limit), 'qc'] = 'in'
    df_gbr_result = pd.concat([df_gbr_train,df_gbr_test])

    metrics_dict = {    'r2_train':r2(y_train, y_pred_train).round(2), 
                        'r2_test': r2(y_test, y_pred_test).round(2),
                        'mae_train':mae(y_train, y_pred_train).round(2), 
                        'mae_test': mae(y_test, y_pred_test).round(2),
                        'test_in': df_gbr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict, 'grid_search':grids_setting_list, 'result_df':df_gbr_result} 
def gbr_train_test_split_iters(train_dataset, target_feature):
    """
    'train_ds', 'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 'grid_search', 'result_df'
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well',target_feature]])
    x = np.array(train_dataset.drop(target_feature, axis=1))
    num = random.randint(0,100)
    train_dataset_list.append(train_dataset.drop(target_feature, axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    y_train_wells = y_train_init[:,0]
    y_test_wells = y_test_init[:,0]
    x_train = x_train_init[:,1:]
    x_test = x_test_init[:,1:]
    y_train = y_train_init[:,1]
    y_test = y_test_init[:,1]
    # GridSearch for ML-model
    grid_gbr = GradientBoostingRegressor()
    grid_param_gbr = {  "n_estimators": [50],
                        "max_depth": [10],
                        "learning_rate": [0.1],
                        "max_features": ["sqrt"],
                        "random_state": [42]}
    gbr = Pipeline([("scaler",StandardScaler()),("gbr",GradientBoostingRegressor(   n_estimators = grid_param_gbr['n_estimators'][0], 
                                                                                    max_depth = grid_param_gbr['max_depth'][0],  
                                                                                    learning_rate = grid_param_gbr['learning_rate'][0], 
                                                                                    max_features = grid_param_gbr['max_features'][0],
                                                                                    random_state = grid_param_gbr['random_state'][0]))]) 
    gbr.fit(x_train, y_train)
    y_pred_train = gbr.predict(x_train)
    y_pred_test = gbr.predict(x_test)
    
    # QC of predicted values for train & test datasets
    df_gbr_train = pd.DataFrame(zip(y_train_wells, y_train, y_pred_train), columns=['well', 'actual','predict'])
    df_gbr_train['l_limit'] = df_gbr_train.actual*0.75
    df_gbr_train['h_limit'] = df_gbr_train.actual*1.25
    df_gbr_train['qc'] = 'out'
    df_gbr_train['dataset'] = 'train'
    df_gbr_train.loc[(df_gbr_train.predict >= df_gbr_train.l_limit) & (df_gbr_train.predict <= df_gbr_train.h_limit), 'qc'] = 'in'
    df_gbr_test = pd.DataFrame(zip(y_test_wells, y_test, y_pred_test), columns=['well', 'actual','predict'])
    df_gbr_test['l_limit'] = df_gbr_test.actual*0.75
    df_gbr_test['h_limit'] = df_gbr_test.actual*1.25
    df_gbr_test['qc'] = 'out'
    df_gbr_test['dataset'] = 'test'
    df_gbr_test.loc[(df_gbr_test.predict >= df_gbr_test.l_limit) & (df_gbr_test.predict <= df_gbr_test.h_limit), 'qc'] = 'in'
    df_gbr_result = pd.concat([df_gbr_train,df_gbr_test])

    metrics_dict = {    'r2_train':r2(y_train, y_pred_train).round(2), 
                        'r2_test': r2(y_test, y_pred_test).round(2),
                        'mae_train':mae(y_train, y_pred_train).round(2), 
                        'mae_test': mae(y_test, y_pred_test).round(2),
                        'test_in': df_gbr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    return {'train_ds':train_dataset_list[0], 'metrics':metrics_dict, 'result_df':df_gbr_result} 

### Data collection

In [23]:
# Selecting wells without outliers
well_clean_8a = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII')].well 
well_clean_8 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII sand')].well
well_clean_10a = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X')].well
well_clean_10 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X sand')].well
well_clean_10_20 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X 20')].well
well_clean_10_40 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany X 40')].well
well_clean_8_25 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 25')].well
well_clean_8_20 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 20')].well
well_clean_8_15 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 15')].well
well_clean_8_10 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 10')].well
well_clean_8_5 = df_khtst_bal_qcl[(df_khtst_bal_qcl.FORMATION == 'Balakhany VIII 5')].well
# Distance calculation
qc_n = 1
dist_bal8a = well_dist_calc('Balakhany VIII', qc_n, well_clean_8a)
dist_bal8 = well_dist_calc('Balakhany VIII sand', qc_n, well_clean_8)
dist_bal10a = well_dist_calc('Balakhany X', qc_n, well_clean_10a)
dist_bal10 = well_dist_calc('Balakhany X sand', qc_n, well_clean_10)
dist_bal10_20 = well_dist_calc('Balakhany X 20', qc_n, well_clean_10_20)
dist_bal10_40 = well_dist_calc('Balakhany X 40', qc_n, well_clean_10_40)
dist_bal8_25 = well_dist_calc('Balakhany VIII 25', qc_n, well_clean_8_25)
dist_bal8_20 = well_dist_calc('Balakhany VIII 20', qc_n, well_clean_8_20)
dist_bal8_15 = well_dist_calc('Balakhany VIII 15', qc_n, well_clean_8_15)
dist_bal8_10 = well_dist_calc('Balakhany VIII 10', qc_n, well_clean_8_10)
dist_bal8_5 = well_dist_calc('Balakhany VIII 5', qc_n, well_clean_8_5)
# Creating datasets according to FU
df_well_kh_dist_bal8a_fld = well_offsets_collection(dist_bal8a, 'Balakhany VIII', df_khtst_xy, metadata)
df_well_kh_dist_bal8_fld = well_offsets_collection(dist_bal8, 'Balakhany VIII sand', df_khtst_xy, metadata)
df_well_kh_dist_bal8_25_fld = well_offsets_collection(dist_bal8_25, 'Balakhany VIII 25', df_khtst_xy, metadata)
df_well_kh_dist_bal8_20_fld = well_offsets_collection(dist_bal8_20, 'Balakhany VIII 20', df_khtst_xy, metadata)
df_well_kh_dist_bal8_15_fld = well_offsets_collection(dist_bal8_15, 'Balakhany VIII 15', df_khtst_xy, metadata)
df_well_kh_dist_bal8_10_fld = well_offsets_collection(dist_bal8_10, 'Balakhany VIII 10', df_khtst_xy, metadata)
df_well_kh_dist_bal8_5_fld = well_offsets_collection(dist_bal8_5, 'Balakhany VIII 5', df_khtst_xy, metadata)
df_well_kh_dist_bal10a_fld = well_offsets_collection(dist_bal10a, 'Balakhany X', df_khtst_xy, metadata)                                   
df_well_kh_dist_bal10_fld = well_offsets_collection(dist_bal10, 'Balakhany X sand', df_khtst_xy, metadata)
df_well_kh_dist_bal10_40_fld = well_offsets_collection(dist_bal10_40, 'Balakhany X 40', df_khtst_xy, metadata)
df_well_kh_dist_bal10_20_fld = well_offsets_collection(dist_bal10_20, 'Balakhany X 20', df_khtst_xy, metadata)

In [ ]:
#Concatenation all dist-datasets                                                  
df_well_kh_dist_all = pd.concat([   df_well_kh_dist_bal8a_fld,
                                    df_well_kh_dist_bal8_fld,    
                                    df_well_kh_dist_bal8_25_fld,
                                    df_well_kh_dist_bal8_20_fld,
                                    df_well_kh_dist_bal8_15_fld,
                                    df_well_kh_dist_bal8_10_fld, 
                                    df_well_kh_dist_bal8_5_fld,
                                    df_well_kh_dist_bal10a_fld,                                    
                                    df_well_kh_dist_bal10_fld,
                                    df_well_kh_dist_bal10_40_fld,
                                    df_well_kh_dist_bal10_20_fld])
df_well_kh_dist_all.shape

In [25]:
#Calculation of TST-thickness Balakhany VIII & X
df_fu_tst = df_prq[(df_prq.FORMATION.str.contains('Balakhany VIII')) | (df_prq.FORMATION.str.contains('Balakhany X'))]
df_fu_tst = df_fu_tst[['well', 'MD','FORMATION','TST']]
df_fu_tst_top = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[0]).reset_index()
df_fu_tst_top.rename(columns={'TST':'TST_top'}, inplace=True)
df_fu_tst_bot = df_fu_tst.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1]).reset_index()
df_fu_tst_bot.rename(columns={'TST':'TST_bot'}, inplace=True)
df_fu_tst_final = df_fu_tst_top.set_index(['well','FORMATION']).join(df_fu_tst_bot.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final['TST_interv'] = round((df_fu_tst_final.TST_bot - df_fu_tst_final.TST_top),0)
#Joining coordinates datasets
df_fu_tst_final = df_fu_tst_final.set_index(['well','FORMATION']).join(xy_coord.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index(['well', 'FORMATION']).join(df_prq_tvdss.set_index(['well','FORMATION'])).reset_index()
df_fu_tst_final = df_fu_tst_final.set_index('well').join(df_prq.groupby('well')['field'].apply(lambda x: x.iloc[0])).reset_index()
df_fu_tst_final = df_fu_tst_final[(df_fu_tst_final.TST_interv > 0)]

In [26]:
#Reading df_prq_htst_avgprop_v1 and getting outliers
path = 'C:\\jupyter\\SPP\\inputoutput\\' 
df_htst_avgprop = pd.read_csv(path + 'df_prq_htst_avgprop_v1.csv')
#Preparation weighted average df_htst_avgprop-dataset
cutoff_h_tst = -1
cutoff_perm_avg = -1
#Applying filtration to dataset with cutoffs
df_htst_avgprop_nz = df_htst_avgprop[(df_htst_avgprop.h_tst > cutoff_h_tst) & (df_htst_avgprop.md_perm_avg > cutoff_perm_avg)]
#Multiplaying htst by resprop values
df_htst_avgprop_nz['kavg_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_perm_avg
df_htst_avgprop_nz['phit_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_phit_avg
df_htst_avgprop_nz['vsh_htst'] = df_htst_avgprop_nz.h_tst * df_htst_avgprop_nz.md_vsh_avg
#Summarizing h_tst via well & formation
df_htst_fm = df_htst_avgprop_nz.groupby(['well','FORMATION'])['h_tst'].sum().reset_index()
df_htst_fm.rename(columns={'h_tst':'res_tst'}, inplace=True)
#Calculating weighted averages
df_htst_avgprop_nz_avgpropsum = df_htst_avgprop_nz.groupby(['well','FORMATION'])[['phit_htst','vsh_htst']].sum().reset_index()
df_htst_avgprop_nz_avgpropsum_join = df_htst_avgprop_nz_avgpropsum.set_index(
                                     ['well','FORMATION']).join(df_htst_fm.set_index(['well','FORMATION'])).reset_index()
df_htst_avgprop_nz_avgpropsum_join['phit_wavg'] = df_htst_avgprop_nz_avgpropsum_join.phit_htst / df_htst_avgprop_nz_avgpropsum_join.res_tst
df_htst_avgprop_nz_avgpropsum_join['vsh_wavg'] = df_htst_avgprop_nz_avgpropsum_join.vsh_htst / df_htst_avgprop_nz_avgpropsum_join.res_tst

In [ ]:
#Calculating avg properties and collecting df per FU
df_8abal_phhpv = df_avg_properties('Balakhany VIII', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_phhpv = df_avg_properties('Balakhany VIII sand', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10abal_phhpv = df_avg_properties('Balakhany X', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_phhpv = df_avg_properties('Balakhany X sand', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_20_phhpv = df_avg_properties('Balakhany X 20', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_10bal_40_phhpv = df_avg_properties('Balakhany X 40', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_25_phhpv = df_avg_properties('Balakhany VIII 25', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_20_phhpv = df_avg_properties('Balakhany VIII 20', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_15_phhpv = df_avg_properties('Balakhany VIII 15', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_10_phhpv = df_avg_properties('Balakhany VIII 10', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
df_8bal_5_phhpv = df_avg_properties('Balakhany VIII 5', df_htst_avgprop_nz, df_htst_avgprop_nz_avgpropsum_join)
#Concatenation of all df per FU
df_all_bal_phhpv = pd.concat([  df_8abal_phhpv, df_8bal_phhpv, df_10abal_phhpv, df_10bal_phhpv, df_10bal_20_phhpv, df_10bal_40_phhpv, 
                                df_8bal_25_phhpv, df_8bal_20_phhpv, df_8bal_15_phhpv, df_8bal_10_phhpv, df_8bal_5_phhpv])
df_all_bal_phhpv_tstint = df_all_bal_phhpv.set_index(['well','FORMATION']).join(df_fu_tst_final.set_index(['well','FORMATION'])).reset_index()
df_avgprop_final_wa = df_all_bal_phhpv_tstint.copy()
df_avgprop_final_wa.shape

In [28]:
# For Shahriyar
perm_cutoff = -1
perm_cutoff_top = 140_000
azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI', 'CHIRAG']
chg_lst = [ 'DWG', 'DDGG', 'WEST CHIRAG']
df_dist_kh_bal_shahriayr_init = df_avgprop_final_wa.set_index(['well','FORMATION']).join(
                                df_well_kh_dist_all.drop('field',axis=1).set_index(['well','FORMATION'])).reset_index()
df_dist_kh_bal_shahriayr_init.rename(columns={'TST_interv':'interv_tst', 'TST_top':'tst_top', 'TST_bot':'tst_bot'}, inplace=True)
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr_init[df_dist_kh_bal_shahriayr_init.MD.notna()]
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr[ (df_dist_kh_bal_shahriayr.kh1>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kh2>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kh3>perm_cutoff) &
                                                     (df_dist_kh_bal_shahriayr.kavg_htst>perm_cutoff)]
df_dist_kh_bal_shahriayr = df_dist_kh_bal_shahriayr[ (df_dist_kh_bal_shahriayr.kh1<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kh2<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kh3<perm_cutoff_top) &
                                                     (df_dist_kh_bal_shahriayr.kavg_htst<perm_cutoff_top)]
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.field.isin(azr_lst), 'fields_zone'] = 'azeri'
df_dist_kh_bal_shahriayr.loc[df_dist_kh_bal_shahriayr.field.isin(chg_lst), 'fields_zone'] = 'chirag'
df_dist_kh_bal_shahriayr['FORMATION_up_orig'] = df_dist_kh_bal_shahriayr['FORMATION_up']
df_dist_kh_bal_shahriayr['kavg_htst_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kavg_htst'])
df_dist_kh_bal_shahriayr['kh1_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh1'])
df_dist_kh_bal_shahriayr['kh2_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh2'])
df_dist_kh_bal_shahriayr['kh3_lg10'] = np.log10(df_dist_kh_bal_shahriayr['kh3'])
df_dist_kh_bal_shahriayr[['X_new', 'Y_new']] = df_dist_kh_bal_shahriayr.apply(lambda row: rotate(row['X'], row['Y']), axis=1, result_type='expand')
df_dist_kh_bal_shahriayr = pd.get_dummies(df_dist_kh_bal_shahriayr, columns = ['FORMATION_up'])
# df_dist_kh_bal_shahriayr.info()

In [29]:
# df_dist_kh_bal_shahriayr_init.to_csv('df_all_features_nofilters.csv', index=False)

## ML-testing

### Xplots depiction

In [30]:
# Dataset generation
df_allf_xplot = df_dist_kh_bal_shahriayr[[    
                                        # 'well', 
                                        'kavg_htst', 
                                        'interv_tst','res_tst', 'vsh_wavg',
                                        'dist1', 'dist2', 'dist3',
                                        'phit_wavg',  
                                        'kh1', 'kh2', 'kh3', 
                                        'X', 'Y', 
                                        'MD', 'TVD_SCS',
                                        # 'FORMATION_up_orig', 'fields_zone', 'field',
                                        # 'x1', 'x2', 'x3', 'y1', 'y2', 'y3',
                                        # 'well1', 'well2', 'well3', 'KHtst', 'kavg_htst', 
                                        # 'kavg_htst_lg10', 
                                        # 'kh1_lg10', 'kh2_lg10', 'kh3_lg10', 'X_new', 'Y_new',
                                        # 'FORMATION_up_Balakhany VIII', 'FORMATION_up_Balakhany X',
                                        ]]
df_allf_xplot = df_allf_xplot[df_allf_xplot.phit_wavg.notna()]

In [ ]:
#Corr heatmap
f, ax = plt.subplots(figsize=(12,8))
ax = sns.heatmap(df_allf_xplot.corr(), annot=True)

In [ ]:
#Pairplot
df_allf_xplot_v2 = df_dist_kh_bal_shahriayr[[    
                                        # 'well',              
                                        'interv_tst','res_tst', 'vsh_wavg',
                                        'dist1', 'dist2', 'dist3',
                                        'phit_wavg',  
                                        'kh1', 'kh2', 'kh3',
                                        'kavg_htst',  
                                        # 'MD', 'TVD_SCS',
                                        'fields_zone',]]
sns.pairplot(df_allf_xplot_v2, hue='fields_zone',plot_kws=dict(alpha=0.5));

In [ ]:
#Xplot
sns.scatterplot(df_allf_xplot_v2, y='res_tst', x='interv_tst', hue='fields_zone');

### 70/30 splits testing polygon for different ML

In [39]:
df_allf = df_dist_kh_bal_shahriayr[[    
                                        'well', 
                                        'kavg_htst', 
                                        'interv_tst','res_tst', 'vsh_wavg',
                                        'dist1', 'dist2', 'dist3',
                                        'fields_zone',
                                        # 'field',
                                        'kh1', 'kh2', 'kh3', 
                                        'X', 'Y', 
                                        'MD', 'TVD_SCS',
                                        # 'FORMATION_up_orig', 
                                        'x1', 'x2', 'x3', 'y1', 'y2', 'y3',
                                        # 'well1', 'well2', 'well3', 'KHtst', 'kavg_htst', 
                                        # 'kavg_htst_lg10', 
                                        # 'kh1_lg10', 'kh2_lg10', 'kh3_lg10', 'X_new', 'Y_new',
                                        'FORMATION_up_Balakhany VIII', 'FORMATION_up_Balakhany X',
                                        'phit_wavg',  
                                        ]]
df_allf = df_allf[df_allf.phit_wavg.notna()]
# df_allf_v2 = df_allf
df_allf_v2 = pd.get_dummies(df_allf, columns = ['fields_zone'])
# df_allf = df_allf[(df_allf.kh1_lg10>0) & (df_allf.kh2_lg10>0) & (df_allf.kh3_lg10>0)]

In [ ]:
#GBR run
report_gbr_list = []
targ_feat = 'kavg_htst'
out_list = ['well', targ_feat]
for i in df_allf_v2.columns[2:]:
    out_list.append(i)
    gbr_iter = gbr_train_test_split_iters(df_allf_v2[out_list], targ_feat)
    report_gbr_list.append((i, gbr_iter['metrics']['test_in'], gbr_iter['metrics']['r2_test']))
report_gbr = pd.DataFrame(report_gbr_list, columns=['feature','qc_in','r2_test'])
fig_rep = px.line(report_gbr, x='feature', y='qc_in')
fig_r2 = px.line(report_gbr, x='feature', y='r2_test')
fig_r2.update_traces(line=dict(color = 'red', dash='dash'))
fig_final = go.Figure(data = fig_rep.data + fig_r2.data)
fig_final.update_layout(title='Step by step adding new feature GBR, red = f(R2), blue = f(predicted_kh)', width=1300,height=400, margin=dict(l=10,r=10,b=10,t=40))

In [ ]:
# #SVR run - need to be repaired!!!
# targ_feat = 'kavg_htst'
# out_list = ['well', targ_feat]
# report_svr_list = []
# for i in df_allf_v2.columns[2:]:
#     out_list.append(i)
#     svr_iter = svr_train_test_split_iter(df_allf_v2[out_list], targ_feat)
#     report_svr_list.append((i, svr_iter['metrics']['test_in'], svr_iter['metrics']['r2_test']))
# report_svr = pd.DataFrame(report_svr_list, columns=['feature','qc_in','r2_test'])
# fig_rep = px.line(report_svr, x='feature', y='qc_in')
# fig_r2 = px.line(report_svr, x='feature', y='r2_test')
# fig_r2.update_traces(line=dict(color = 'red', dash='dash'))
# fig_final = go.Figure(data = fig_rep.data + fig_r2.data)
# fig_final.update_layout(title='Step by step adding new feature SVR, red = f(R2), blue = f(predicted_kh)', width=1300,height=400, margin=dict(l=10,r=10,b=10,t=40))

In [ ]:
#RFR run
targ_feat = 'kavg_htst'
out_list = ['well', targ_feat]
report_rfr_list = []
for i in df_allf_v2.columns[2:]:
    out_list.append(i)
    rfr_iter = rfr_train_test_split_iter(df_allf_v2[out_list], targ_feat)
    report_rfr_list.append((i, rfr_iter['metrics']['test_in'], rfr_iter['metrics']['r2_test']))
report_rfr = pd.DataFrame(report_rfr_list, columns=['feature','qc_in', 'r2_test'])
fig_rep = px.line(report_rfr, x='feature', y='qc_in')
fig_r2 = px.line(report_rfr, x='feature', y='r2_test')
fig_r2.update_traces(line=dict(color = 'red', dash='dash'))
fig_final = go.Figure(data = fig_rep.data + fig_r2.data)
fig_final.update_layout(title='Step by step adding new feature RFR, red = f(R2), blue = f(predicted_kh)', width=1300,height=400, margin=dict(l=10,r=10,b=10,t=40))

In [ ]:
# # Permutation importance train set & test sets
# result_pi_train = permutation_importance(gbr, x_train, y_train, n_repeats=20, random_state=num, n_jobs=2)
# sorted_importances_idx_train = result_pi_train.importances_mean.argsort()
# importances_train = pd.DataFrame(result_pi_train.importances[sorted_importances_idx_train].T,
#                                 columns=df_dist_kh_bal_shahriayr_final.drop(['well','kavg_htst'], axis=1).columns[sorted_importances_idx_train])
# fig, ax = plt.subplots(1,2, figsize=(15,5))
# ax[0].boxplot(importances_train, labels=importances_train.columns[sorted_importances_idx_train], vert=False, whis=10)
# ax[0].set_title("Permut Imp (train set)")
# ax[0].axvline(x=0, color="k", linestyle="--")
# ax[0].set_xlabel("Decrease in accuracy score")
# # Permutation importance test set
# result_pi_test = permutation_importance(gbr, x_test, y_test, n_repeats=20, random_state=num, n_jobs=2)
# sorted_importances_idx_test = result_pi_test.importances_mean.argsort()
# importances_test = pd.DataFrame(result_pi_test.importances[sorted_importances_idx_test].T,
#                            columns=df_dist_kh_bal_shahriayr_final.drop(['well','kavg_htst'], axis=1).columns[sorted_importances_idx_test])
# ax[1].boxplot(importances_test, labels=importances_test.columns[sorted_importances_idx_test], vert=False, whis=10)
# ax[1].set_title("Permut Imp (test set)")
# ax[1].axvline(x=0, color="k", linestyle="--")
# ax[1].set_xlabel("Decrease in accuracy score")
# ax[1].figure.tight_layout()
# fig.tight_layout()

In [ ]:
# Making up the x-plot for train dataset

# df_gbr_total = pd.concat([df_gbr_train, df_gbr_test])
# print('wells total:', df_gbr_train.shape[0])
# print('wells unpredicted:', df_gbr_train['qc'].value_counts()['out'], (df_gbr_train['qc'].value_counts()['out']/df_gbr_train.shape[0]).round(3), 'v/v')
# print('wells predicted:', df_gbr_train['qc'].value_counts()['in'], (df_gbr_train['qc'].value_counts()['in']/df_gbr_train.shape[0]).round(3), 'v/v')
# max_val = 14000
# fig1_ml = px.scatter(gbr_split_notphit['result_df'][gbr_split_notphit['result_df']['dataset']=='train'], x='actual', y='predict', 
#                      color='qc', 
#                      hover_data=['well'], 
#                      width=800, height=400,
#                      color_discrete_sequence=["green", "red"])
# fig1_ml.update_traces(marker=dict(size=10,opacity=0.75,line=dict(color='rgb(47, 57, 61)', width=1)))
# fig2_ml=px.line(x=[0,max_val], y=[0,max_val])
# fig2_1_ml=px.line(x=[0,max_val], y=[0,max_val*1.25])
# fig2_2_ml=px.line(x=[0,max_val], y=[0,max_val*0.75])
# fig2_ml.update_traces(line=dict(color = 'blue'))
# fig2_1_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig2_2_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig3_ml = go.Figure(data = fig1_ml.data + fig2_ml.data + fig2_1_ml.data + fig2_2_ml.data)
# fig3_ml.update_layout(title = 'Comparison Actual vs Pred Train',width=600,height=400, xaxis_title='actual', yaxis_title='predict',
#                       margin=dict(l=10,r=10,b=10,t=40))
# fig3_ml.show()

In [ ]:
# # Making up the x-plot for test dataset
# fig1_ml = px.scatter(gbr_split_notphit['result_df'][gbr_split_notphit['result_df']['dataset']=='test'], x='actual', y='predict', 
#                      color='qc', 
#                      hover_data=['well'], 
#                      width=400, height=400,
#                      color_discrete_sequence=["red", "green"])
# fig1_ml.update_traces(marker=dict(size=10,opacity=0.75,line=dict(color='rgb(47, 57, 61)', width=1)))
# fig2_ml=px.line(x=[0,max_val], y=[0,max_val])
# fig2_1_ml=px.line(x=[0,max_val], y=[0,max_val*1.25])
# fig2_2_ml=px.line(x=[0,max_val], y=[0,max_val*0.75])
# fig2_ml.update_traces(line=dict(color = 'blue'))
# fig2_1_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig2_2_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig3_ml = go.Figure(data = fig1_ml.data + fig2_ml.data + fig2_1_ml.data + fig2_2_ml.data)
# fig3_ml.update_layout(title = 'Comparison Actual vs Pred',width=600,height=400, xaxis_title='actual', yaxis_title='predict',
#                       margin=dict(l=10,r=10,b=10,t=40))

### Run GBR

In [41]:
# # Starting of the loop
# y_test_lst = []
# y_pred_test_lst = []
# well_exclude_lst = []
# gs_settings_lst = []
# metrics_r2_mae_lst = []
# df_dist_kh_bal_shahriayr_gbr = df_allf_v2.sample(frac = 1).reset_index().drop('index', axis=1)
# for i in tqdm(range(len(df_dist_kh_bal_shahriayr_gbr))):
#     #Making up the feature and target datasets
#     df_wo_well = df_dist_kh_bal_shahriayr_gbr.drop([i])
#     well_exclude = df_dist_kh_bal_shahriayr_gbr.iloc[i]['well']
#     well_exclude_lst.append(well_exclude)
#     y_train = np.array(df_wo_well['kavg_htst_lg10'])
#     x_train = np.array(df_wo_well[[ 'X', 'Y', 'TVD_SCS', 'interv_tst', 'res_tst', 'vsh_wavg',
#                                     'kh1_lg10', 'kh2_lg10', 'kh3_lg10',
#                                     'FORMATION_up_Balakhany VIII', 'FORMATION_up_Balakhany X']])
#     well_train = np.array(df_wo_well['well'])
#     y_test = np.array(df_dist_kh_bal_shahriayr_gbr.iloc[i]['kavg_htst_lg10'])
#     y_test_lst.append(y_test)
#     x_test = np.array(df_dist_kh_bal_shahriayr_gbr.iloc[i][[    'X', 'Y', 'TVD_SCS', 'interv_tst', 'res_tst', 'vsh_wavg',
#                                                                 'kh1_lg10', 'kh2_lg10', 'kh3_lg10',
#                                                                 'FORMATION_up_Balakhany VIII', 'FORMATION_up_Balakhany X']])
#     # GridSearch for ML-model
#     # {'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 90}
#     grid_param_GBR = {
#                         "n_estimators": [100],
#                         "max_depth": [10],
#                         "learning_rate": [0.1],
#                         "max_features": ["sqrt"],
#                         'random_state': [90]
#                                     }
#     GS_setting = grid_param_GBR
#     gs_settings_lst.append((GS_setting['n_estimators'],
#                             GS_setting['max_depth'], 
#                             GS_setting['learning_rate'],
#                             GS_setting['max_features'],
#                             GS_setting['random_state']))
#     # Statement of ML-model
#     gbr = Pipeline([("scaler",StandardScaler()),("gbr",GradientBoostingRegressor(   n_estimators = GS_setting['n_estimators'][0], 
#                                                                                     max_depth = GS_setting['max_depth'][0],  
#                                                                                     learning_rate = GS_setting['learning_rate'][0], 
#                                                                                     max_features = GS_setting['max_features'][0],
#                                                                                     random_state = GS_setting['random_state'][0]))])
#     # Fitting the ML-model
#     gbr.fit(x_train, y_train)
#     y_pred_train = gbr.predict(x_train)
#     y_pred_test = gbr.predict([x_test])
#     # Block of data naturalization
#     # y_pred_test_nat = sc_y.inverse_transform(y_pred_test.reshape(-1, 1))
#     y_pred_test_lst.append(y_pred_test[0])
#     # Metrics computation for the ML-model
#     r2_train = r2(y_train, y_pred_train).round(5)
#     mae_train = mae(y_train, y_pred_train)
#     # mae_train_nat = sc_y.inverse_transform([[mae_train]])
#     metrics_r2_mae_lst.append((r2_train, mae_train.round(5)))

# # Building up of dataframe
# res_gbr_sha = pd.DataFrame(zip(y_test_lst,y_pred_test_lst,well_exclude_lst, gs_settings_lst), 
#                    columns = ['test','predict','well', 'gs_setting',])
# res_gbr_sha.test = res_gbr_sha.test.astype('float')
# res_gbr_sha['l_test'] = np.log10((10**res_gbr_sha.test)*0.75)
# res_gbr_sha['h_test'] = np.log10((10**res_gbr_sha.test)*1.25)
# res_gbr_sha['qc'] = 'out'
# res_gbr_sha.loc[(res_gbr_sha.predict >= res_gbr_sha.l_test) & (res_gbr_sha.predict <= res_gbr_sha.h_test), 'qc'] = 'in'
# print('wells total:', res_gbr_sha.shape[0])
# print('wells unpredicted:', res_gbr_sha['qc'].value_counts()['out'], (res_gbr_sha['qc'].value_counts()['out']/res_gbr_sha.shape[0]).round(3), 'v/v')
# print('wells predicted:', res_gbr_sha['qc'].value_counts()['in'], (res_gbr_sha['qc'].value_counts()['in']/res_gbr_sha.shape[0]).round(3), 'v/v')
# mae_df_xy = mae(10**res_gbr_sha.test, 10**res_gbr_sha.predict).round(5)
# r2_df_xy = r2(res_gbr_sha.test, res_gbr_sha.predict).round(0)
# print('mae:', mae_df_xy, 'mDm')
# print('r2:', r2_df_xy)

### Reporting

In [ ]:
# # Making up the final x-plot
# max_val = np.log10(20000)
# fig1_ml = px.scatter(res_gbr_sha, x='test', y='predict', 
#                      color='qc', 
#                      hover_data=['well'], 
#                      width=400, height=400,
#                      color_discrete_sequence=["green", "red"])
# fig1_ml.update_traces(marker=dict(size=10,opacity=0.75,line=dict(color='rgb(47, 57, 61)', width=1)))
# fig2_ml=px.line(x=[0,max_val], y=[0,max_val])
# fig2_1_ml=px.line(x=[0,max_val], y=[0,np.log10((10**max_val)*1.25)])
# fig2_2_ml=px.line(x=[0,max_val], y=[0,np.log10((10**max_val)*0.75)])
# fig2_ml.update_traces(line=dict(color = 'blue'))
# fig2_1_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig2_2_ml.update_traces(line=dict(color = 'blue', dash='dash'))
# fig3_ml = go.Figure(data = fig1_ml.data + fig2_ml.data + fig2_1_ml.data + fig2_2_ml.data)
# fig3_ml.update_layout(title = 'Comparison Actual vs Pred Shahriyar GBR',width=600,height=400, xaxis_title='test', yaxis_title='predict',
#                       margin=dict(l=10,r=10,b=10,t=40), xaxis_type="log", yaxis_type="log")